# TO DO
1. extract all text untill next example
2. check for empty content section of extracted examples


# bugs:
1. how to classify examples with only numbers and material names (maybe there can only be actual??)

# Imports

In [7]:
from collection import collect_urls,download_files,unzip_files
from utils import extract_examples_start_w_word,find_doc_number,process_siblings,extract_num_dot_examples,extract_examples_w_word,extract_experiments_w_heading,save_as_json,remove_duplicate_docs,download_files,fetch_urls_from_pto,get_latest_versions,process_xml_files,read_xml_file,extract_classify_num_patents_w_experiments,extract_ipc_dic,extract_classify_num_patents_w_experiments_w_subclass

download_path = "D:\\patents_data_23_24"
unzip_path = "D:\\unzipped_patents_23_24"

In [ ]:
di = extract_ipc_dic(),"INFORMATION AND COMMUNICATION TECHNOLOGY [ICT] SPECIALLY ADAPTED FOR SPECIFIC APPLICATION FIELDS"

In [ ]:
G16,H10

[]

In [26]:
di["G"]["G16"]

'INFORMATION AND COMMUNICATION TECHNOLOGY [ICT] SPECIALLY ADAPTED FOR SPECIFIC APPLICATION FIELDS'

In [24]:
keys = [key for key, value in di["H"].items() if value in ["SEMICONDUCTOR DEVICES; ELECTRIC SOLID-STATE DEVICES NOT OTHERWISE PROVIDED FOR","INFORMATION AND COMMUNICATION TECHNOLOGY [ICT] SPECIALLY ADAPTED FOR SPECIFIC APPLICATION FIELDS"]]
keys

['H10']

In [3]:
import asyncio
import os
import aiofiles
from lxml import etree
from collections import defaultdict

# Assuming these functions exist and are correct
# from your_module import extract_ipc_dic, extract_experiments_w_heading, save_as_json

async def process_file(file_path, ipc_dic, subclass_dict, lock):
    """
    Process each XML file asynchronously, updating subclass_dict once file is fully processed.
    """
    all_xml_parts = []
    try:
        async with aiofiles.open(file_path, mode="r", encoding="utf-8") as f:
            content = await f.read()
            parts = content.split('<?xml version="1.0" encoding="UTF-8"?>')
            parts = [p for p in parts if p.strip()]
            all_xml_parts.extend(parts)
    except Exception as e:
        print(f"Error processing {file_path}: {str(e)}")
        return

    total_num_of_patents = len(all_xml_parts)
    for j, xml in enumerate(all_xml_parts):
        if j % 2000 == 0:
            print(f"Processing {j}/{total_num_of_patents}")

        root = etree.fromstring(xml.encode(), etree.XMLParser(recover=True))
        try:
            sections = root.xpath("//classifications-ipcr/classification-ipcr/section")
            classes = root.xpath("//classifications-ipcr/classification-ipcr/class")

            # Get unique subclass names for this patent
            this_subclass_names = set()
            for section_elem, class_elem in zip(sections, classes):
                section = section_elem.text
                class_code = class_elem.text
                full_code = section + class_code

                try:
                    subclass_name = ipc_dic[section][full_code]
                    this_subclass_names.add(subclass_name)
                except KeyError:
                    continue

            # Lock the update to subclass_dict to ensure thread-safety
            async with lock:
                for subclass_name in this_subclass_names:
                    if subclass_name not in subclass_dict:
                        subclass_dict[subclass_name] = {
                            "with_examples": 0,
                            "without_examples": 0,
                        }

                    if extract_experiments_w_heading(xml):
                        subclass_dict[subclass_name]["with_examples"] += 1
                    else:
                        subclass_dict[subclass_name]["without_examples"] += 1

        except IndexError:
            continue

async def extract_classify_num_patents_w_experiments_w_subclass(
    folder_path="D:\\unzipped_patents_23_24",
):
    ipc_dic = extract_ipc_dic()
    subclass_dict = defaultdict(lambda: {"with_examples": 0, "without_examples": 0})
    lock = asyncio.Lock()

    file_names = os.listdir(folder_path)
    total_num_of_patents = 0
    tasks = []
    for i, file in enumerate(file_names):
        if file.endswith(".xml"):
            print(f"Scheduling {file}... ({i + 1}/{len(file_names)})")
            file_path = os.path.join(folder_path, file)
            tasks.append(process_file(file_path, ipc_dic, subclass_dict, lock))

    # Wait for all tasks to complete
    await asyncio.gather(*tasks)

    total_num_of_patents = sum(len(file.split('<?xml version="1.0" encoding="UTF-8"?>')) for file in os.listdir(folder_path))
    
    # Optional: Save the final dictionary to a JSON file
    start = file_names[0].split("a")[1].split(".")[0] if len(file_names) > 0 else "start"
    end = file_names[-1].split("a")[1].split(".")[0] if len(file_names) > 0 else "end"

    save_as_json(
        dict(subclass_dict),
        f"subclass_of_{total_num_of_patents}_Patents_{start}_{end}.json",
    )

    return dict(subclass_dict)

# To run the async function
# asyncio.run(extract_classify_num_patents_w_experiments_w_subclass(folder_path))


In [3]:
7530+5861+7445+6774+6809

34419

In [2]:
ola = extract_classify_num_patents_w_experiments_w_subclass(folder_path = "D:/prophetic_patents/unzipped_patents_2015/test")

Processing ipa150101.xml... (1/5)
Processing 0/7530
Processing 2000/7530
Processing 4000/7530
Processing 6000/7530
Saved as subclass_of_7530_Patents_150101_150101.json
Processing ipa150108.xml... (2/5)
Processing 0/5861
Processing 2000/5861
Processing 4000/5861
Saved as subclass_of_13391_Patents_150101_150108.json
Processing ipa150115.xml... (3/5)
Processing 0/7445
Processing 2000/7445
Processing 4000/7445
Processing 6000/7445
Saved as subclass_of_20836_Patents_150101_150115.json
Processing ipa150122.xml... (4/5)
Processing 0/6774
Processing 2000/6774
Processing 4000/6774
Processing 6000/6774
Saved as subclass_of_27610_Patents_150101_150122.json
Processing ipa150129.xml... (5/5)
Processing 0/6809
Processing 2000/6809
Processing 4000/6809
Processing 6000/6809
Saved as subclass_of_34419_Patents_150101_150129.json


In [5]:
ola = await extract_classify_num_patents_w_experiments_w_subclass(folder_path="E:\\unzipped_patents_23_24\\test") 

Scheduling ipa240104.xml... (1/5)
Scheduling ipa240111.xml... (2/5)
Scheduling ipa240118.xml... (3/5)
Scheduling ipa240125.xml... (4/5)
Scheduling ipa240201.xml... (5/5)
Processing 0/8708
Processing 2000/8708
Processing 4000/8708
Processing 6000/8708
Processing 8000/8708
Processing 0/8726
Processing 2000/8726
Processing 4000/8726
Processing 6000/8726
Processing 8000/8726
Processing 0/9298
Processing 2000/9298
Processing 4000/9298
Processing 6000/9298
Processing 8000/9298
Processing 0/7959
Processing 2000/7959
Processing 4000/7959
Processing 6000/7959
Processing 0/7715
Processing 2000/7715
Processing 4000/7715
Processing 6000/7715
Saved as subclass_of_5_Patents_240104_240201.json


C:\Users\am3081\AppData\Local\Temp\ipykernel_16360\976412090.py:1: RuntimeWarning: coroutine 'extract_classify_num_patents_w_experiments_w_subclass' was never awaited
  ola = await extract_classify_num_patents_w_experiments_w_subclass(folder_path="E:\\unzipped_patents_23_24\\test")


In [10]:
save_as_json(ola,"subclass_of_42406_Patents_240104_240201.json")

Saved as subclass_of_42406_Patents_240104_240201.json


In [7]:
import pandas as pd
df = pd.read_json("subclass_of_34419_Patents_150101_150129.json").T
df.rename(columns={"examples":"with_examples"},inplace=True)
totals_row = pd.DataFrame({
    'with_examples': [df["with_examples"].sum()],
    'without_examples': [df["without_examples"].sum()],
    
}, index=['Totals(Actual number of patents in last coloumn)'])
df = pd.concat([df, totals_row])
df["Total(Actual 34419)"] = df["with_examples"] + df["without_examples"]
df.to_excel("D://prophetic_patents/subclass_of_34419_Patents_150101_150129.xlsx")

In [5]:
total_examples = sum(d['examples'] for d in ola.values())
total_without = sum(d['without_examples'] for d in ola.values())
print(total_examples+total_without)
len(ola)

6628


115

# Data Collection

In [2]:
# url ="https://bulkdata.uspto.gov/data/patent/application/redbook/fulltext/2023/"
# rp = requests.get(url, timeout=10)
# parser = etree.XMLParser(recover=True)
# root = etree.fromstring(rp.text.encode(), parser)
# href_values = root.findall(".//a[@href]") 
# urls = [href.get('href') for href in href_values if href.get('href').endswith('.zip')]
# for ur in urls[:10]:
#     print(url + ur)

start_year = int(input("Enter the year you want to start from: "))
end_year = int(input("Enter the year you want to end at: "))
# download_path = "E:\\patents_data_23_24"
# unzip_path = "E:\\unzipped_patents_23_24"

urls = fetch_urls_from_pto(start_year,end_year)
print(len(urls))


latest_versions = get_latest_versions(urls)

files = []
for year in range(start_year, end_year + 1):
        url = f"https://bulkdata.uspto.gov/data/patent/application/redbook/fulltext/{year}/"
        files.append(download_files(url, download_path, latest_versions[year]))


2
Downloaded ipa230105.zip ------- 1 / 52
Downloaded ipa230112_r1.zip ------- 2 / 52
Downloaded ipa230119_r1.zip ------- 3 / 52
Downloaded ipa230126_r2.zip ------- 4 / 52
Downloaded ipa230202_r2.zip ------- 5 / 52
Downloaded ipa230209_r1.zip ------- 6 / 52
Downloaded ipa230216_r2.zip ------- 7 / 52
Downloaded ipa230223.zip ------- 8 / 52
Downloaded ipa230302.zip ------- 9 / 52
Downloaded ipa230309.zip ------- 10 / 52
Downloaded ipa230316.zip ------- 11 / 52
Downloaded ipa230323.zip ------- 12 / 52
Downloaded ipa230330.zip ------- 13 / 52
Downloaded ipa230406.zip ------- 14 / 52
Downloaded ipa230413.zip ------- 15 / 52
Downloaded ipa230420.zip ------- 16 / 52
Downloaded ipa230427.zip ------- 17 / 52
Downloaded ipa230504_r1.zip ------- 18 / 52
Downloaded ipa230511_r1.zip ------- 19 / 52
Downloaded ipa230518.zip ------- 20 / 52
Downloaded ipa230525.zip ------- 21 / 52


KeyboardInterrupt: 

In [ ]:
# sectors_dict = extract_classify_num_patents_w_experiments(unzip_path)
# save_as_json(sectors_dict, "sectors_dict.json")
# """{'Human Necessities': {'examples': 6672, 'without_examples': 26723},
#  'Performing Operations; Transporting': {'examples': 1579,
#   'without_examples': 22461},
#  'Chemistry; Metallurgy': {'examples': 10363, 'without_examples': 8122},
#  'Textiles; Paper': {'examples': 194, 'without_examples': 774},
#  'Fixed Constructions': {'examples': 85, 'without_examples': 4141},
#  'Mechanical Engineering; Lighting; Heating': {'examples': 168,
#   'without_examples': 9716},
#  'Physics': {'examples': 2232, 'without_examples': 58046},
#  'Electricity': {'examples': 2364, 'without_examples': 57086},
#  'Unknown Sector': {'examples': 0, 'without_examples': 0},
#  'Not Found': {'examples': 0, 'without_examples': 6280},
#  'Total_num_of_patents': 217006,
#  'Total_num_of_patents_w_examples': 23657,
#  'Total_num_of_patents_without_examples': 193349}
# """

{'Human Necessities': {'examples': 6672, 'without_examples': 26723},
 'Performing Operations; Transporting': {'examples': 1579,
  'without_examples': 22461},
 'Chemistry; Metallurgy': {'examples': 10363, 'without_examples': 8122},
 'Textiles; Paper': {'examples': 194, 'without_examples': 774},
 'Fixed Constructions': {'examples': 85, 'without_examples': 4141},
 'Mechanical Engineering; Lighting; Heating': {'examples': 168,
  'without_examples': 9716},
 'Physics': {'examples': 2232, 'without_examples': 58046},
 'Electricity': {'examples': 2364, 'without_examples': 57086},
 'Unknown Sector': {'examples': 0, 'without_examples': 0},
 'Not Found': {'examples': 0, 'without_examples': 6280},
 'Total_num_of_patents': 217006,
 'Total_num_of_patents_w_examples': 23657,
 'Total_num_of_patents_without_examples': 193349}

In [12]:
total_without_examples = sum(sector['without_examples'] for sector in sectors_dict.values())
total_with_examples = sum(sector['examples'] for sector in sectors_dict.values())
print(f"Total number of documents without examples: {total_without_examples}, with examples: {total_with_examples}")
for sector, values in sectors_dict.items():
    print(f"{sector}: {values['examples']} with examples, {values['without_examples']} without examples")

Total number of documents without examples: 53724, with examples: 6220
Human Necessities: 1702 with examples, 7467 without examples
Performing Operations; Transporting: 444 with examples, 6423 without examples
Chemistry; Metallurgy: 2743 with examples, 2158 without examples
Textiles; Paper: 47 with examples, 237 without examples
Fixed Constructions: 25 with examples, 1169 without examples
Mechanical Engineering; Lighting; Heating: 51 with examples, 2910 without examples
Physics: 587 with examples, 15829 without examples
Electricity: 621 with examples, 15914 without examples
Unknown Sector: 0 with examples, 0 without examples
Not Found: 0 with examples, 1617 without examples


# Processing

In [2]:
xml_parts = process_xml_files(unzip_path + "\\test")

In [15]:
len(xml_parts)

8047

In [ ]:
from lxml import etree
with open("unzipped_patents_data/ipa230105.xml", "r", encoding="utf-8") as file:
    content = file.read()
xml_parts = content.split('<?xml version="1.0" encoding="UTF-8"?>')
#xml_parts = read_xml_file("E:\\2023-092024.xml")

In [3]:
# with open("2023-092024.xml", "w", encoding="utf-8") as file:
#         for part in xml_parts:
#             file.write(part)
#             file.write("\n")

In [2]:
# with open("unzipped_patents_data\\ipa230105.xml", "r", encoding="utf-8") as file:
#     content = file.read()
# xml_parts = content.split('<?xml version="1.0" encoding="UTF-8"?>')

In [3]:
xml_no_dup = remove_duplicate_docs(xml_parts)
print(f"Number of duplicates {len(xml_parts) - len(xml_no_dup)} out of {len(xml_parts)}")

Number of duplicates 224 out of 8047


In [3]:
# from bs4 import BeautifulSoup

# def extract_experiments_w_heading(text):
#     """Extracts the 'Examples' section and its experiments from a patent text."""

#     # Use BeautifulSoup to parse the structure (for HTML-like patents)
#     soup = BeautifulSoup(text, "html.parser")

#     # Find the "EXAMPLES" section heading
#     examples_heading = soup.find(
#         lambda tag: tag.name == "heading"
#         and (
#             "EXAMPLES" in tag.text.upper()
#             or "EXAMPLE" == tag.text.upper()
#             or "EXPERIMENT" == tag.text.upper()
#             or "EXPERIMENTS" in tag.text.upper()
#         )
#     )
#     if not examples_heading:
#         # print("No 'Examples' section found.")
#         return None


#     return examples_heading 

In [26]:
a = extract_experiments_w_heading(xml_parts[28760])
process_siblings(a[0].find_next_siblings())

[{'number': 'Example 1',
  'title': '',
  'content': ['This example describes one of the many potential sets of criteria for the selection of sites of incorporation of non-naturally encoded amino acids into relaxin.',
   'FIGS. 1-4 show the structure and the sequence of relaxin and the table below includes sequences with the A chain, B chain, relaxin and prorelaxin. Relaxin polypeptides were generated by substituting a naturally encoded amino acid with a non-naturally encoded amino acid. Each polypeptide had one of the amino acids substituted with para-acetylphenylalanine (pAcF or pAF). The polypeptides generated lacked the leader sequence and were A/B chain relaxin polypeptides (SEQ ID NO. 1-3). Each of the polypeptides generated had a non-naturally encoded amino acid substitution at one of the following positions 1, 5, 18, 13, 2 of SEQ ID NO: 4 or in those positions of the A chain of any of the known relaxin sequences or 5, 7, 18, 28 of SEQ ID NO: 5 or 6 in those same positions of th

In [5]:
def extract_experiments_from_docs(xml_docs, max_docs=3000):
    """
    Extracts experiment sections from XML documents.
    
    Parameters:
        xml_docs (list): List of XML documents to process.
        max_docs (int): Maximum number of documents to process (default: 3000).
        
    Returns:
        dict: Dictionary mapping document numbers to extracted experiment sections.
    """
    doc_w_exp = {}
    
    for i, xml in enumerate(xml_docs[:max_docs], start=1):
        if i % 1000 == 0:
            print(f"{i}/{len(xml_docs)} so far found {len(doc_w_exp)} docs with experiments")
        
        heading = extract_experiments_w_heading(xml)
        if heading:
            extracted_examples = []
            
            if len(heading) > 1:
                print(f"has more than 1 Examples heading, {i}")
            
            elif len(heading) == 1:
                example_start_w = process_siblings(heading[0].find_next_siblings())
                if example_start_w:
                    if len(example_start_w[0]["content"]) == 0:
                        extracted_ex_w_word = extract_examples_start_w_word(heading[0].find_next_siblings())
                        if isinstance(extracted_ex_w_word, list):
                            if len(extracted_ex_w_word[0]["content"]) > 0:
                                extracted_examples.append(extracted_ex_w_word)
                        elif len(extracted_ex_w_word["content"]) > 0:
                            extracted_examples.append(extracted_ex_w_word)
                    else:
                        extracted_examples.append(example_start_w)
                else:
                    extracted_ex_w_word = extract_examples_start_w_word(heading[0].find_next_siblings())
                    if extracted_ex_w_word and len(extracted_ex_w_word["content"]) > 0:
                        extracted_examples.append(extracted_ex_w_word)
                    else:
                        num_dot_examples = extract_num_dot_examples(str(heading[0].find_next_siblings()))
                        if len(num_dot_examples) > 0:
                            extracted_examples.append(num_dot_examples)
            
            else:
                extracted_ex_w_word = extract_examples_w_word(xml)
                if extracted_ex_w_word and len(extracted_ex_w_word["content"]) > 0:
                    extracted_examples.append(extracted_ex_w_word)
                else:
                    example_start_w = process_siblings(heading[0].find_next_siblings())
                    if example_start_w and len(example_start_w[0]["content"]) > 0:
                        extracted_examples.append(example_start_w)
            
            if extracted_examples:
                doc_w_exp[int(find_doc_number(xml)[0])] = extracted_examples
    
    return doc_w_exp


In [16]:

doc_w_exp = {}
i = 0
for i,xml in enumerate(xml_no_dup[:3000]):
    i += 1
    if i % 1000 == 0:
        print(f"{i}/{len(xml_no_dup)} so far found {len(doc_w_exp)} docs with experiments")
    heading = extract_experiments_w_heading(xml)
    if heading:
        extracted_examples = []
        if len(heading) >1:
            print(f"has more than 1 Examples heading , {i}")
            # for example in heading:
            #     extracted_ex_start_w_word = process_siblings(example.find_next_siblings())
            #     if extracted_ex_start_w_word and len(extracted_ex_start_w_word) > 0:
            #         extracted_examples.append(extracted_ex_start_w_word)
            #     else:
            #         extracted_ex_w_word = extract_examples_start_w_word(example.find_next_siblings())
            #         if extracted_ex_w_word:
            #             if len(extracted_ex_w_word) > 0:
            #                 extracted_examples.append(extracted_ex_w_word)
            #         else:
            #             num_dot_examples = extract_num_dot_examples(str(heading[0].find_next_siblings()))
            #             if len(num_dot_examples) > 0:
            #                 extracted_examples.append(num_dot_examples)
        elif len(heading) == 1:
            example_start_w = process_siblings(heading[0].find_next_siblings())
            if example_start_w:
                if len(example_start_w[0]["content"]) == 0:
                    extracted_ex_w_word = extract_examples_start_w_word(heading[0].find_next_siblings())
                    try:
                        if len(extracted_ex_w_word["content"]) > 0:
                            extracted_examples.append(extracted_ex_w_word)
                    except:
                        if len(extracted_ex_w_word[0]["content"]) > 0:
                            extracted_examples.append(extracted_ex_w_word)
                else:
                    extracted_examples.append(example_start_w)
            else:
                extracted_ex_w_word = extract_examples_start_w_word(heading[0].find_next_siblings())
                if extracted_ex_w_word:
                    if len(extracted_ex_w_word["content"]) > 0:
                        extracted_examples.append(extracted_ex_w_word)
                else:
                    num_dot_examples = extract_num_dot_examples(str(heading[0].find_next_siblings()))
                    if len(num_dot_examples) > 0:
                        extracted_examples.append(num_dot_examples)
        else:
            extracted_ex_w_word = extract_examples_w_word(xml)
            if extracted_ex_w_word:
                if len(extracted_ex_w_word["content"]) > 0:
                    extracted_examples.append(extracted_ex_w_word)
                else:
                    example_start_w = process_siblings(heading[0].find_next_siblings())
                    if example_start_w:
                        if len(example_start_w[0]["content"]) > 0:
                            extracted_examples.append(example_start_w)

            # else:
            #     num_dot_examples = extract_num_dot_examples(str(heading[0].find_next_siblings()))
            #     extracted_examples.append(num_dot_examples)
    
        
    
        if len(extracted_examples) > 0:
            doc_w_exp[int(find_doc_number(xml)[0])] = extracted_examples
            # if len(doc_w_exp) == 100:
            #     break


"""100/8048 so far found 17 docs with experiments
200/8048 so far found 30 docs with experiments
300/8048 so far found 33 docs with experiments
400/8048 so far found 38 docs with experiments
500/8048 so far found 42 docs with experiments
600/8048 so far found 45 docs with experiments
700/8048 so far found 46 docs with experiments
800/8048 so far found 90 docs with experiments
900/8048 so far found 146 docs with experiments
1000/8048 so far found 187 docs with experiments
1100/8048 so far found 233 docs with experiments
1200/8048 so far found 251 docs with experiments
1300/8048 so far found 258 docs with experiments
1400/8048 so far found 263 docs with experiments
1500/8048 so far found 294 docs with experiments
1600/8048 so far found 298 docs with experiments
1700/8048 so far found 302 docs with experiments
1800/8048 so far found 320 docs with experiments
1900/8048 so far found 326 docs with experiments
2000/8048 so far found 329 docs with experiments
2100/8048 so far found 331 docs with experiments
2200/8048 so far found 332 docs with experiments
2300/8048 so far found 334 docs with experiments
2400/8048 so far found 361 docs with experiments
2500/8048 so far found 435 docs with experiments
with dup: 1196   out of 8048 in 12 minutes"""

"""100/7824 so far found 20 docs with experiments
200/7824 so far found 34 docs with experiments
300/7824 so far found 36 docs with experiments
400/7824 so far found 42 docs with experiments
500/7824 so far found 45 docs with experiments
600/7824 so far found 49 docs with experiments
700/7824 so far found 51 docs with experiments
800/7824 so far found 98 docs with experiments
900/7824 so far found 162 docs with experiments
1000/7824 so far found 217 docs with experiments
1100/7824 so far found 253 docs with experiments
1200/7824 so far found 261 docs with experiments
1300/7824 so far found 266 docs with experiments
1400/7824 so far found 295 docs with experiments
1500/7824 so far found 304 docs with experiments
1600/7824 so far found 309 docs with experiments
1700/7824 so far found 328 docs with experiments
1800/7824 so far found 333 docs with experiments
1900/7824 so far found 339 docs with experiments
2000/7824 so far found 342 docs with experiments
2100/7824 so far found 343 docs with experiments
2200/7824 so far found 345 docs with experiments
2300/7824 so far found 358 docs with experiments
2400/7824 so far found 439 docs with experiments
2500/7824 so far found 510 docs with experiments
no dup 949  out of 7824 in 13.5 minutes on own laptop


4537/46862 in 39min on uni pc 
2432 /30085 with examples etc being the only thing in the heading when fiding examples section
2370 /29405  without duplicates with extract_examples_start_w_word first """

has more than 1 Examples heading , 356
has more than 1 Examples heading , 695
has more than 1 Examples heading , 786
has more than 1 Examples heading , 851
1000/7823 so far found 179 docs with experiments
2000/7823 so far found 275 docs with experiments
has more than 1 Examples heading , 2270
has more than 1 Examples heading , 2320
has more than 1 Examples heading , 2388
has more than 1 Examples heading , 2408
3000/7823 so far found 602 docs with experiments


'100/7824 so far found 20 docs with experiments\n200/7824 so far found 34 docs with experiments\n300/7824 so far found 36 docs with experiments\n400/7824 so far found 42 docs with experiments\n500/7824 so far found 45 docs with experiments\n600/7824 so far found 49 docs with experiments\n700/7824 so far found 51 docs with experiments\n800/7824 so far found 98 docs with experiments\n900/7824 so far found 162 docs with experiments\n1000/7824 so far found 217 docs with experiments\n1100/7824 so far found 253 docs with experiments\n1200/7824 so far found 261 docs with experiments\n1300/7824 so far found 266 docs with experiments\n1400/7824 so far found 295 docs with experiments\n1500/7824 so far found 304 docs with experiments\n1600/7824 so far found 309 docs with experiments\n1700/7824 so far found 328 docs with experiments\n1800/7824 so far found 333 docs with experiments\n1900/7824 so far found 339 docs with experiments\n2000/7824 so far found 342 docs with experiments\n2100/7824 so far

In [18]:
save_as_json(doc_w_exp, "olallalal.json")

Saved as olallalal.json


In [6]:
from classify_patent import check_tense_nltk
def dic_to_dic_w_tense(doc_w_exp):
    dic = {}

    for key, value in doc_w_exp.items():
        dic_inner = {}
        if isinstance(value[0], list) and len(value[0]) == 1:
            example = value[0][0]["number"]
            desc = value[0][0]["title"] + "." + "".join(value[0][0]["content"])
            dic[key] = {example: [check_tense_nltk(desc),desc]}


        elif isinstance(value[0], list) and len(value[0]) > 1:
            i = 1
            for ls in value[0]:
                example = ls["number"]
                desc = ls["title"] + "." + "".join(ls["content"])
                dic_inner[i] = {example: [check_tense_nltk(desc),desc]}
                i += 1
            
            dic[key] = dic_inner   
        elif isinstance(value[0], dict):
            j = 1
            for ex,description in value[0].items():
                dic_inner[j] = {ex: [check_tense_nltk(description),description]}
                j += 1
            dic[key] = dic_inner
    return dic

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\amoha\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\amoha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [22]:
test_dic = dic_to_dic_w_tense(doc_w_exp)

In [23]:
print(f"{len(test_dic)} , {len(doc_w_exp)}")

602 , 602


In [6]:
# from classify_patent import check_tense_nltk

# dic = {}

# for doc_num, exps in doc_w_exp.items():
#     if not exps:
#         continue  # Skip if exps is empty

#     # Check if the first element is a dictionary
#     if isinstance(exps[0], dict):
#         dic[doc_num] = {}  # Initialize empty dict for this doc

#         for exp in exps:  # Iterate over list items
#             if isinstance(exp, dict):
#                 for key, val in exp.items():  # Corrected key-value order
#                     dic[doc_num][key] = check_tense_nltk(val)

#     # Check if the first element is a list
#     elif isinstance(exps[0], list) and len(exps[0]) > 1:
#         dic[doc_num] = {}  # Initialize empty dict for this doc

#         for exp in exps:
#             if isinstance(exp, dict):
#                 for key, val in exp.items():
#                     dic[doc_num][key] = check_tense_nltk(val)
#             else:
#                 # Ensure index safety
#                 if len(exps) > 1:
#                     dic[doc_num][exps[0]] = check_tense_nltk(exps[1])

#     # Default case if `exps` is a mixed list
#     else:
#         dic[doc_num] = {}  # Initialize empty dict for this doc

#         for exp in exps:
#             if isinstance(exp, dict):
#                 for key, val in exp.items():
#                     dic[doc_num][key] = check_tense_nltk(val)
#             else:
#                 if len(exps) > 1:
#                     dic[doc_num][exps[0]] = check_tense_nltk(exps[1])

from classify_patent import check_tense_nltk

dic = {}

def process_expressions(doc_num, exps, dic):
    """ Recursively process lists and dictionaries in exps """
    if doc_num not in dic:
        dic[doc_num] = {}

    for exp in exps:
        if isinstance(exp, dict):  
            for key, val in exp.items():
                if isinstance(val, str):  # Ensure val is a string
                    dic[doc_num][key] = check_tense_nltk(val)
                elif isinstance(val, list):  # If it's a list, process it recursively
                    process_expressions(doc_num, val, dic)

        elif isinstance(exp, list):  # Handle nested lists
            process_expressions(doc_num, exp, dic)  # Recursively process lists

        elif isinstance(exp, str):  # Handle individual string elements
            dic[doc_num][exp] = check_tense_nltk(exp)

# Iterate through main dictionary
for doc_num, exps in doc_w_exp.items():
    if exps:  # Ensure exps is not empty
        process_expressions(doc_num, exps, dic)


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\AM3081\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\AM3081\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
save_as_json(doc_w_exp,filename="no-dup_testing_example_exatly_in_heading_siblingfirst.json")

Saved as no-dup_testing_example_exatly_in_heading_siblingfirst.json


In [11]:
import requests
import os

def collect_urls(year):
    url = f"https://bulkdata.uspto.gov/data/patent/application/redbook/fulltext/{year}/"
    # params = {
    #     "data": f'{{"name":"APPXML","fromDate":"{start_date}","toDate":"{end_date}"}}'
    # }
    print(url)
    response = requests.get(url, timeout=10)
    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        return []
    #print(f"Urls collected for the period from {start_date} to {end_date} successfully")
    return response.json()["productFiles"]

# start_date = input("Enter start date YYYY-MM: ")
# end_date = input("Enter end date YYYY-MM: ")
year = input("Enter year: ")
download_path = "patents_data"
unzip_path = "unzipped_patents_data"

try:
    os.makedirs(download_path, exist_ok=True)
    print(f"Download path set to: {download_path}")
except PermissionError:
    print("Error: No permission to create directory on D drive")
    # Fallback to current directory
    download_path = "patents_data"
    os.makedirs(download_path, exist_ok=True)
    print(f"Fallback: Download path set to: {download_path}")

files = collect_urls(year)
if files:
    total_size = sum(file["fileSize"] / 1024 / 1024 for file in files)
    print(
        f"\nTotal file size: {round(total_size / 1024, 2)} GB for  {year}\n"
    )
    download_files(files, download_path)
    unzip_files(download_path, unzip_path)
    

In [ ]:

df = read_xlsb_file()

In [4]:
df[(df.issueyear == 2015) | (df.prophetic !=0)][["patentnumber","prophetic","nonprophetic","issueyear"]]

,patentnumber,prophetic,nonprophetic,issueyear
1,3998932.0,1.0,1.0,1974.0
7065,3988434.0,1.0,3.0,1976.0
7066,3967989.0,1.0,2.0,1976.0
7067,3941773.0,1.0,2.0,1976.0
7068,3939058.0,1.0,5.0,1976.0
...,...,...,...,...
559401,9597310.0,70.0,0.0,2017.0
559402,9670166.0,84.0,28.0,2017.0
559403,9662339.0,92.0,4.0,2017.0
559404,9643983.0,141.0,3.0,2017.0


In [60]:
import os
path = "unzipped_patents_2015"
dic = {}
for file in os.listdir(path):
    if file.endswith(".xml"):
        content = read_xml_file(os.path.join(path, file))
        parts = content.split('<?xml version="1.0" encoding="UTF-8"?>')[1:]
        for part in parts:
            doc_num = find_doc_number(part)
            patent_num = find_patent_number(part)
            if doc_num and patent_num:
                dic[patent_num[0]] = doc_num[0]

XMLSyntaxError: None (<string>, line 0)

In [59]:
import pprint as pp

pp.pprint(response.text)

('<!DOCTYPE html>\n'
 '<html lang="en">\n'
 '<head>\n'
 '    <meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />\n'
 '    <title>US-20230000022-A1 - Patent Public Search | USPTO</title>\n'
 '    <link href="/dirsearch-public/css/patents.css" rel="stylesheet" />\n'
 '</head>\n'
 '<body>\n'
 '<div id="container">\n'
 '    <header class="bottom-border padding">\n'
 '    <div class="grid-container-max-auto bottom-border">\n'
 '        <div id="logo" class="grid-col-start-1 padding-right"></div>\n'
 '        <h1>US Patent & Trademark Office<br> Patent Public Search | Text '
 'View</h1>\n'
 '    </div>\n'
 '    <div class="grid-container-max-auto padding-top-only">\n'
 '        <span class="grid-col-start-1 grid-col-1 text-top">United States '
 '<span id="category">Patent Application Publication</span> </span>\n'
 '        <span class="grid-col-1 text-top text-right"><span '
 'class="MatchFromOtherFieldStyle">20230000022</span></span>\n'
 '        <span class="grid-col-star

In [56]:
import requests
patent_number = 20230000022
print(patent_number)
url = f"https://ppubs.uspto.gov/dirsearch-public/patents/html/{patent_number}?source=USPAT&requestToken=eyJzdWIiOiI3MjVhZmViMy05MWIwLTRkZjQtYTI2OC1mMjQ0NThkZWQxMzQiLCJ2ZXIiOiJhNjNmYWE5OS04ZDcyLTRlMGEtYjExMS0wNzA5MGE3YWM0MTMiLCJleHAiOjB9"
response = requests.get(url)

heading = extract_experiments_w_heading(response.text)
if heading:
    extracted_examples = []
    
    if len(heading) > 1:
        print(f"has more than 1 Examples heading, {i}")
    
    elif len(heading) == 1:
        example_start_w = process_siblings(heading[0].find_next_siblings())
        if example_start_w:
            if len(example_start_w[0]["content"]) == 0:
                extracted_ex_w_word = extract_examples_start_w_word(heading[0].find_next_siblings())
                if isinstance(extracted_ex_w_word, list):
                    if len(extracted_ex_w_word[0]["content"]) > 0:
                        extracted_examples.append(extracted_ex_w_word)
                elif len(extracted_ex_w_word["content"]) > 0:
                    extracted_examples.append(extracted_ex_w_word)
            else:
                extracted_examples.append(example_start_w)
        else:
            extracted_ex_w_word = extract_examples_start_w_word(heading[0].find_next_siblings())
            if extracted_ex_w_word and len(extracted_ex_w_word["content"]) > 0:
                extracted_examples.append(extracted_ex_w_word)
            else:
                num_dot_examples = extract_num_dot_examples(str(heading[0].find_next_siblings()))
                if len(num_dot_examples) > 0:
                    extracted_examples.append(num_dot_examples)
    
    else:
        extracted_ex_w_word = extract_examples_w_word(xml)
        if extracted_ex_w_word and len(extracted_ex_w_word["content"]) > 0:
            extracted_examples.append(extracted_ex_w_word)
        else:
            example_start_w = process_siblings(heading[0].find_next_siblings())
            if example_start_w and len(example_start_w[0]["content"]) > 0:
                extracted_examples.append(example_start_w)
    
    if extracted_examples:
        doc_w_exp[int(find_doc_number(xml)[0])] = extracted_examples
doc_w_exp

20230000022


NameError: name 'doc_w_exp' is not defined

In [2]:
year = 2015
download_path = "E:\\patents_2015"
url = f"https://bulkdata.uspto.gov/data/patent/application/redbook/fulltext/{year}/"
ulrs_2015 =  fetch_urls_from_pto(year,year)
latest_versions = get_latest_versions(ulrs_2015)
download_files(url, "patents_2015", latest_versions[year])
unzip_files("patents_2015", "unzipped_patents_2015")

Downloaded ipa150101.zip ------- 1 / 53
Downloaded ipa150108.zip ------- 2 / 53
Downloaded ipa150115.zip ------- 3 / 53
Downloaded ipa150122.zip ------- 4 / 53
Downloaded ipa150129.zip ------- 5 / 53
Downloaded ipa150205.zip ------- 6 / 53
Downloaded ipa150212.zip ------- 7 / 53
Downloaded ipa150219.zip ------- 8 / 53
Downloaded ipa150226.zip ------- 9 / 53
Downloaded ipa150305.zip ------- 10 / 53
Downloaded ipa150312.zip ------- 11 / 53
Downloaded ipa150319.zip ------- 12 / 53
Downloaded ipa150326.zip ------- 13 / 53
Downloaded ipa150402.zip ------- 14 / 53
Downloaded ipa150409.zip ------- 15 / 53
Downloaded ipa150416.zip ------- 16 / 53
Downloaded ipa150423.zip ------- 17 / 53
Downloaded ipa150430.zip ------- 18 / 53
Downloaded ipa150507.zip ------- 19 / 53
Downloaded ipa150514.zip ------- 20 / 53
Downloaded ipa150521.zip ------- 21 / 53
Downloaded ipa150528.zip ------- 22 / 53
Downloaded ipa150604.zip ------- 23 / 53
Downloaded ipa150611.zip ------- 24 / 53
Downloaded ipa150618.zip 

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'E:\\patents_2015'

In [3]:
unzip_files("patents_2015", "unzipped_patents_2015")

Unzipping files:   2%|▏         | 1/53 [00:07<06:20,  7.32s/it]

Unzipped ipa150101.zip to unzipped_patents_2015


Unzipping files:   4%|▍         | 2/53 [00:12<05:21,  6.30s/it]

Unzipped ipa150108.zip to unzipped_patents_2015


Unzipping files:   6%|▌         | 3/53 [00:20<05:45,  6.92s/it]

...

Unzipped ipa151217.zip to unzipped_patents_2015


Unzipping files:  98%|█████████▊| 52/53 [06:22<00:07,  7.95s/it]

Unzipped ipa151224.zip to unzipped_patents_2015


Unzipping files: 100%|██████████| 53/53 [06:31<00:00,  7.38s/it]

Unzipped ipa151231.zip to unzipped_patents_2015
